In [1]:
# %%cmd
# pip install flask pandas numpy matplotlib scikit-learn jieba zhconv

In [2]:
# %%cmd
# pip list

# 基于朴素贝叶斯的中文文本情感分析

## python 库版本
list
|   Package    | Version |
| :----------: | :-----: |
|    Flask     |  2.0.2  |
|    jieba     | 0.42.1  |
|  matplotlib  |  3.4.3  |
|    numpy     | 1.21.4  |
|    pandas    |  1.3.4  |
| scikit-learn |  1.0.1  |
|    scipy     |  1.7.2  |

## 文本处理
1. 去除停用词，标点符号等
2. 文章参考 https://zhuanlan.zhihu.com/p/39437488
3. 字段含义 https://github.com/SophonPlus/ChineseNlpCorpus/blob/master/datasets/simplifyweibo_4_moods/intro.ipynb

| 字段   | 说明                           |
| :----- | :----------------------------- |
| label  | 0 喜悦，1 愤怒，2 厌恶，3 低落 |
| review | 微博内容                       |

In [3]:
import pandas as pd
import jieba
import zhconv
import re

In [4]:
# 36万多条，带情感标注 新浪微博，包含 4 种情感，其中喜悦约 20 万条，愤怒、厌恶、低落各约 5 万条
emotion = {0: '喜悦', 1: '愤怒', 2: '厌恶', 3: '低落'}
df = pd.read_csv(r'.\data\simplifyweibo_4_moods.csv')
df.sample(20)

,label,review
211372,1,一个师弟介绍了一个他的师兄给我认识。。跟陌生人聊天不习惯。~ 谁告诉我一个好的FLASH 话...
167340,0,她去广东进货了也。。。也好，散散心。。。
253585,2,加油灏明！在心里默默为你祈祷~ ~ ~ 希望你能赶紧好起来！大家也快一起帮灏明加油吧！天娱官...
303993,2,试试嘿嘿是不是哦？？试一下据说，只要这个就能求到自己想要的真爱不信你试试。
171854,0,回复他的小名叫大辉我本来叫他大新他听了生气了逼着我叫他大辉我不想输给他就加了狼哈哈为什么叫大...
169146,0,曝！OPPO U525 内置新浪微博！只要关注我，本条微博并评论为：，加玩围脖心得，就有机会...
242652,1,在天朝，但凡出了人命，就是升官发财的机会！！基本上升官了，一级就是一条人命
263082,2,妞妞这几天好似不舒服，连拉了三天肚子，想想距离上次吃打虫药时间正好三个月了，于是今天中午飞车...
137689,0,呵呵，哥们太有才了，支持~ ~ x 谢您欣赏！好一个粉丝耶，难渡汪洋海。众多粉丝哟，新浪开大...
211566,1,云南文山壮族苗族自治州麻栗坡县原县委书记、文山州民政局原局长赵仕永贪污受贿近500万，还有千...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361744 entries, 0 to 361743
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   361744 non-null  int64 
 1   review  361744 non-null  object
dtypes: int64(1), object(1)
memory usage: 5.5+ MB


In [6]:
df["label"].value_counts()

0    199496
2     55267
3     55267
1     51714
Name: label, dtype: int64

In [7]:
# 简繁切换
df["review"] = df["review"].apply(lambda x: zhconv.convert(str(x), 'zh-cn'))

In [8]:
# 读取停用词，stopwords.txt 使用 空格 划分每一个词语
with open("stopwords.txt", "r", encoding="utf-8") as fp:
    stop_word = fp.read().split()
stop_word[: 10]

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*']

In [9]:
def jieba_cut_text(text, stop_word, cut_by=" "):
    # [\u4E00-\u9FA5]+ 去除非中文字符
    patt = re.compile("[\u4E00-\u9FA5]+", re.VERBOSE|re.IGNORECASE)
    word_list = []
    for t in patt.findall(text):
        word_list += [word for word in jieba.cut(t) if word not in stop_word]

    return cut_by.join(word_list)

In [10]:
jieba_cut_text(df.review[339090], stop_word)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\11353\AppData\Local\Temp\jieba.cache
Loading model cost 0.624 seconds.
Prefix dict has been built successfully.


'姐姐 提醒 冥冥之中 画皮 花木兰 锦衣卫 纠葛 前世 今生 恩怨 情仇 前生 佩蓉 欠 勇哥 情 来世 变 作 大漠 中 痴痴 等候 青龙 乔花 前世 王生 欠 佩蓉 债 今生 做 木兰 背后 文泰'

In [11]:
df["review"] = df["review"].apply(lambda x: jieba_cut_text(x, stop_word))

In [12]:
df["review"].sample(20)

143333          回复 晚上 老婆 放假 约 哥们 喝酒 发个 坐标 噢 办公室 很近 中关村 请 吃饭
131902    帽子 帽子 哈哈哈 哈哈哈哈 宝宝 措样 好像 外星人 哈哈哈哈 哈哈哈哈 帽子 戴久 睡着...
60639                                             相机 借到 疯哥谢
319934    岁月流逝 感伤 戳 泪点 年 孩子 演唱会 微博 直播 感动 感谢 十年 前 歌迷 感谢 加...
300495                        搜索 心 触目惊心 繁殖 狗 真实 内幕 请以 领养 买卖
321669                   赶紧 明儿 奔 超市 表唱 咿呀 咿呀 滴 心脏 受不鸟 咿呀 咿呀
327849    回复 真得 资本 挤出 认 最怕 大哥 挤走 企业 老板 想法 一是 企业 资产 卖掉 移民...
268416    等待 我要 里庄 这一课 补上 参与 快乐 片段 收齐 未来 快乐 传递 东东 枪 童鞋 听...
209182    朵花 带回 北京 公道 人心 周笔畅 想 挑拨 笔亲 周笔 之间 关系 想 伤害 孩子 说 ...
54030     近日 德国 著名 街头 艺术家 埃德加 穆勒 花 天 时间 莫斯科 一条 街道 上画 一幅 ...
250994               不说 旁边 那位 施罗德 烦 布什 挺 可爱 突袭 太 囧 讨厌 乔治 布什
20362     今明两天 我市 中到大雨 局部 阵雨 阴雨 驾车 朋友 请 小心 驾驶 天雨路 滑 车速 过...
148695                                              喜欢 百度 汪
216479    杭州 房子 住 元 需 不吃不喝 年 杭州 均价 元 平米 人均收入 元 需 年 北京 均价...
131416    回复 聪明 做法 提起 东西 沉睡 驳倒 马克思 难事 喜好 定义 名词 马克思 必败 管它...
355333    三件 四件 两个 快快 几年 时间 岁 美好 件 事 做 好多 岁 美好 件 事 做 恋爱 ...
328051                           新疆 行 结束

In [14]:
df.head()

,label,review
0,0,啊呀呀 要死 穿 外套 件 余 周小伦 喜欢 五角星 项 链 露 胸 衣服 伦伦 真的 越来越 帅
1,0,大姚 通知 收到 姚 外星人 入侵 地球 摧毁 网络 地球 完蛋 恐惧 越 越 阿粥 外星人...
2,0,风格 喜欢 喜欢 哪张
3,0,试试 去死皮 面膜 燕麦片 加水 中 浸泡 小时 木瓜 牛奶 搅拌 放入 压缩 纸 面膜 敷...
4,0,张老师 谢谢 侬 信任 粉丝 无所谓 重在 质地 近日 发现 现象 他加 关注 回加 取消 ...


In [ ]:
df["label"] = df["label"].apply(lambda x: emotion[x])

In [13]:
df.to_pickle("simplifyweibo_4_moods_preprocessing.pkl.bz2")